In [1]:
def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    import numpy as np
    import pandas as pd
    import plotly.express as px
    import pickle,random
    import plotly.graph_objects as go
    
    data_df=data_df.loc[data_df['Stressor'].notnull()]
    user_df=data_df.loc[data_df.user==user_id]
    user_df=user_df.loc[user_df.user_generated==0]
    with open('/home/jupyter/sneupane/MOODS/pickled_files/colors.pickle', 'rb') as handle:
        colors = pickle.load(handle)
    with open('/home/jupyter/sneupane/MOODS/pickled_files/day_hour_minutes.pickle', 'rb') as handle:
        watch_worn = pickle.load(handle)
    random.shuffle(colors)
    if user_id in watch_worn:
        users_ppg_mins=watch_worn[user_id]
    if len(user_df)>0:
        with open('/home/jupyter/sneupane/MOODS/pickled_files/stressor_abbr.pickle', 'rb') as handle:
            stressor_lemma = pickle.load(handle)
        fig=go.Figure()
        user_df["start_hour"]=user_df["starttime"].dt.hour
        user_df["end_hour"]=user_df["endtime"].dt.hour
#         print(user_df.Stressor.unique())
        user_df_top_stressors=user_df.groupby(["Stressor"],as_index=False)["counter"].count().sort_values("counter",ascending=False).head(5).Stressor.values.tolist()
        user_df=user_df.loc[user_df["Stressor"].isin(user_df_top_stressors)]
       

        
    
        stressors_data =pd.DataFrame([])
        for index, row in user_df.iterrows():
            if row["start_hour"] == row["end_hour"]:
                stressors_data = stressors_data.append(pd.DataFrame({'Stressor': row["Stressor"],'Start Date': row["episode_start_day"],'end Date': row["episode_end_day"],'start_hour':row["starttime"],'end_hour':row["endtime"],
                                                                         'duration':row["duration"],
                                                                       'Total ppg minutes':users_ppg_mins[row["episode_start_day"]][row["start_hour"]],

                                                                      }, 
                                                                      index=[0]), ignore_index=True)

            elif row["start_hour"] != row["end_hour"] :
                    if row["episode_start_day"] == row["episode_end_day"]:
                        total_ppg_minutes=users_ppg_mins[row["episode_start_day"]][row["start_hour"]]\
                                            +users_ppg_mins[row["episode_end_day"]][row["end_hour"]]+ 60*(row["end_hour"] -row["start_hour"]-1)
                        stressors_data = stressors_data.append(pd.DataFrame({'Stressor': row["Stressor"],'Start Date': row["episode_start_day"],'end Date': row["episode_end_day"],
                                                                             'start_hour':row["starttime"],'end_hour':row["endtime"],
                                                                                      'duration':row["duration"],
                                                                               'Total ppg minutes':total_ppg_minutes,

                                                                              }, 
                                                                              index=[0]), ignore_index=True)
                    else:

                        total_ppg_minutes=users_ppg_mins[row["episode_start_day"]][row["start_hour"]] \
                                            +users_ppg_mins[row["episode_end_day"]][row["end_hour"]]+ 60*(row["end_hour"] -row["start_hour"])%24

                        stressors_data = stressors_data.append(pd.DataFrame({'Stressor': row["Stressor"],'Start Date': row["episode_start_day"],'end Date': row["episode_end_day"],'start_hour':row["starttime"],'end_hour':row["endtime"],
                                                                                  'duration':row["duration"],
                                                                               'Total ppg minutes':total_ppg_minutes,

                                                                              }, 
                                                                              index=[0]), ignore_index=True)
    
    

        stressors_data["Duration/mins"]=stressors_data['duration']/stressors_data['Total ppg minutes']*60
        count=1
        for stressor in user_df_top_stressors:
            fig.add_trace(go.Violin(x=stressors_data['Stressor'][stressors_data['Stressor'] == stressor],
                                    y=stressors_data['Duration/mins'][stressors_data['Stressor'] == stressor],
                                    name=stressor,text=stressor,line_color='black',
                                    box_visible=True,fillcolor=colors[count],
                                    meanline_visible=True))
            count+=1
        fig.update_traces(meanline_visible=True,
                          points='all', # show all points
                          jitter=0.05,  # add some jitter on points for better visibility
                          scalemode='count')
        fig.update_yaxes(title_text='Duration of each stress episode (mins per hour)')
        fig.update_traces(showlegend=True)
        fig.update_layout(
                    font_family="Courier New",
                    font_color="blue",
                    title_font_family="Times New Roman",
                    title_font_color="red",
                    legend_title_font_color="green"
                )
        fig.update_yaxes(tickfont_size=10)
        fig.layout.xaxis.showticklabels =False
        fig.update_traces(showlegend=True,spanmode="manual",span=[0,60],hoveron="violins+points", hoverinfo="y",hovertext="text",)
        y_modified_labels=stressor_lemma[user_id]
        

        fig.update_xaxes(ticktext=[y_modified_labels[i] for i in user_df.Stressor.values],tickmode="array",tickvals=user_df.Stressor.values)
        fig.update_layout(legend=dict(yanchor="bottom",title="",
                               orientation="h",y=1.2, xanchor="right",x=1,            
                                        itemclick="toggleothers",
                                        itemdoubleclick="toggle"),
                    autosize=True,

                    margin=dict(
                       l=60,
                            r=30,
                            b=40,
                            t=60,

                    ),)   
        fig.update_layout(hoverlabel=dict(
                                    bgcolor="white",
                                    font_size=16,
                                    font_family="Rockwell",
                                    bordercolor="Black"
                                ),
                          font=dict(
        family="Courier New, monospace",
        size=10,
        color="Black"
    ))
        data_summary_user=data_summary_user_current["Duration Analysis"]    
        
        return "Duration Analysis",fig,data_summary_user
    else:
        return "","",""